# Exploring Space with SQL in Juypter
Goals of this notebook are to see intermediate SQL queries with:
* join statement examples
* window functions

All code is in Python or PostgreSQL.

### Preliminaries
We import some necessary moduels.

In [1]:
import sqlite3
import pandas as pd

Connect to a database.

In [2]:
conn = sqlite3.connect('../data/astronomy_data.db')

Tack a curser to the connection to read the data.

In [3]:
cur = conn.cursor()

### Data about the data
There are four tables of interest in this database:

1. The table called `black_holes` has characteristics of black holes, namely, the name and mass.
      * The unit of mass for each record in this table is in Solar masses. To read the mass in this table, use scientific notation: base_mass x 10^power. For example, the Sombrero galaxy black hole is 1x10⁹ times the mass of the Sun where the `base_mass` is 1 and the `power` is 9.
  
  
2. The table called `solar_system_20` is a relational database table which has facts about the 20 largest objects in the Solar System. It has the following columns:
    * `ss_name` - the unique name of the Solar System object
    * `diameter` - the equatorial width of the object (in miles)
    * `classification` - the type of object in the record (like star or planet)
    * `ss_location` - the location of the object in the Solar System
   
   
3. The table `circ_consts` has information for five constellations that are in the circumpolar region of the sky. It has two columns:
    * `const_name` — the name of the circumpolar constellation
    * `area` — which represents the area of the sky that the boundaries of the constellations enclose, in degrees


4.  The table called `sum_consts`, has information for six constellations that can be seen after sunset in the (Nothern Hemisphere’s) summer. It also has two columns:
    * `const_name` — the name of the summer constellation
    * `main_stars` — how many main stars make up the constellation. These are (usually) the brightest stars in the constellation and are connected in images of the stick figures.
    
    
### Verifying the database works
Let's use some simple `SELECT` queries to see the database in action.

In [4]:
# Select and display all the data in a pandas dataframe.
query = """
        SELECT *
        FROM solar_system_20
        LIMIT 5;
        """
cur.execute(query)


df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]  # for column names
df

,num,ss_name,diameter,classification,ss_location
0,1,Sun,865000,star,close to the center of the Solar System
1,2,Jupiter,88846,planet,5th planet from the Sun
2,3,Saturn,74900,planet,6th planet from the Sun
3,4,Uranus,31763,planet,7th planet from the Sun
4,5,Neptune,30779,planet,8th planet from the Sun


In [5]:
query = """
        SELECT *
        FROM black_holes;
        """
cur.execute(query)


df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]  # for column names
df

,num,name,base_mass,power
0,1,M104 - Sombrero Galaxy,1,9
1,2,M31 - Andromeda Galaxy,1,8
2,3,NGC 4889,1,10
3,4,Cygnus X-1,15,0


### Join statements
Ok. Let's run some join statments.

In [6]:
# full outer join

query = """SELECT *
FROM circ_consts AS c
INNER JOIN sum_consts AS s ON c.const_name = s.const_name;
"""
cur.execute(query)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]  # for column names
df

,const_name,area,const_name,main_stars
0,Cassiopeia,598,Cassiopeia,5
1,Ursa Major,1280,Ursa Major,20


In [7]:
# left outer join

query = """SELECT c.const_name, area, main_stars
FROM circ_consts AS c
LEFT OUTER JOIN sum_consts AS s ON c.const_name = s.const_name;
"""
cur.execute(query)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]  # for column names
df

,const_name,area,main_stars
0,Cassiopeia,598,5.0
1,Ursa Major,1280,20.0
2,Ursa Minor,256,NaN
3,Draco,1083,NaN
4,Cepheus,588,NaN


### A Window function example

In [8]:
query = """SELECT ss_name
    , diameter
    , classification
    , SUM(diameter) OVER(PARTITION BY classification) AS total_class
    , CAST( CAST(diameter AS REAL) / 
            CAST( SUM(diameter) OVER(PARTITION BY classification) AS REAL)  
            * 100 AS INTEGER) AS pct
FROM solar_system_20
ORDER BY classification,
    5 DESC;
  """
cur.execute(query)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]  # for column names
df

,ss_name,diameter,classification,total_class,pct
0,Pluto,1473,dwarf planet,2918,50
1,Eris,1445,dwarf planet,2918,49
2,Ganymede,3270,moon,19440,16
3,Titan,3200,moon,19440,16
4,Callisto,2996,moon,19440,15
5,Io,2264,moon,19440,11
6,The Moon,2159,moon,19440,11
7,Europa,1940,moon,19440,9
8,Triton,1680,moon,19440,8
9,Titana,982,moon,19440,5
